In [1]:
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.12.1+cu113
True


In [ ]:
import os
a = os.listdir("/content/drive/MyDrive/similar/class/class8")
print(len(a))

200


In [ ]:
! nvidia-smi

Thu Sep  8 12:19:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install mmcv -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 563 kB 5.0 MB/s 
     |████████████████████████████████| 190 kB 63.0 MB/s 
  Created wheel for mmcv: filename=mmcv-1.6.1-py2.py3-none-any.whl size=860296 sha256=85d6977d5be92ffb8ad65884090480b2d01e936cc3f18886f056b084917cf4d5
  Stored in directory: /root/.cache/pip/wheels/e0/43/68/40160e8aa085d474903f0ad3764bac92e698936bfcf8a5454b
Successfully built mmcv


In [3]:
"# !git clone https://github.com/open-mmlab/mmclassification.git
%cd /content/drive/MyDrive/mmclassification/

SyntaxError: ignored

In [ ]:
import mmcls
print(mmcls.__version__)

0.23.2


In [ ]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/mmclassification
  Running setup.py develop for mmcls


In [ ]:
! ls configs/mobilenet_v2/mobilenet-v2_8xb32_in1k.py

configs/mobilenet_v2/mobilenet-v2_8xb32_in1k.py


In [ ]:
config_file = 'configs/vgg/vgg16_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/vgg/vgg16_batch256_imagenet_20210208-db26f1a5.pth'

In [ ]:
import mmcv
from mmcls.apis import inference_model, init_model, show_result_pyplot

In [ ]:
device = 'cuda'

In [ ]:
# Build the model according to the config file and load the checkpoint.
model = init_model(config_file, checkpoint_file, device=device)

load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/vgg/vgg16_batch256_imagenet_20210208-db26f1a5.pth


Downloading: "https://download.openmmlab.com/mmclassification/v0/vgg/vgg16_batch256_imagenet_20210208-db26f1a5.pth" to /root/.cache/torch/hub/checkpoints/vgg16_batch256_imagenet_20210208-db26f1a5.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
# The model's inheritance relationship
model.__class__.__mro__

(mmcls.models.classifiers.image.ImageClassifier,
 mmcls.models.classifiers.base.BaseClassifier,
 mmcv.runner.base_module.BaseModule,
 torch.nn.modules.module.Module,
 object)

In [ ]:
# Download the cats & dogs dataset
!wget https://www.dropbox.com/s/wml49yrtdo53mie/cats_dogs_dataset_reorg.zip?dl=0 -O cats_dogs_dataset.zip
!mkdir -p data
!unzip -qo cats_dogs_dataset.zip -d ./data/

--2022-08-30 00:44:38--  https://www.dropbox.com/s/wml49yrtdo53mie/cats_dogs_dataset_reorg.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/wml49yrtdo53mie/cats_dogs_dataset_reorg.zip [following]
--2022-08-30 00:44:38--  https://www.dropbox.com/s/raw/wml49yrtdo53mie/cats_dogs_dataset_reorg.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc67496244b7011a769177afe556.dl.dropboxusercontent.com/cd/0/inline/Br_3zZiOlG0bIQMjX1rea4wMKOeJLr_FTxgoQHnTYz1hzZs2Zlv6v9PXAdHgZycPhst5gpgciXa-nEZk17IY6xRz3v5a1Gn4m8ZBaEXlayxXoR_BLQbnDYziykQqFarFq6-0iHN2aRx_Q5_cEdvRH8PY4Iszrr_iQ3OdtcITKNtGGw/file# [following]
--2022-08-30 00:44:39--  https://uc67496244b7011a769177afe556.dl.dropboxusercontent.com/cd/0/inline/Br_3zZiOlG0bIQMjX1rea4wMKOeJL

In [ ]:
%%writefile configs/mobilenet_v2/mobilenet_v2_1x_custom.py
_base_ = [
    '../_base_/models/mobilenet_v2_1x.py',
    '../_base_/schedules/imagenet_bs256_epochstep.py',
    '../_base_/default_runtime.py'
]

# ---- Model configs ----
# Here we use init_cfg to load pre-trained model.
# In this way, only the weights of backbone will be loaded.
# And modify the num_classes to match our dataset.

model = dict(
    backbone=dict(
        init_cfg = dict(
            type='Pretrained', 
            checkpoint='https://download.openmmlab.com/mmclassification/v0/mobilenet_v2/mobilenet_v2_batch256_imagenet_20200708-3b2dc3af.pth', 
            prefix='backbone')
    ),
    head=dict(
        num_classes=5,
        topk = (1, )
    ))

# ---- Dataset configs ----
# We re-organized the dataset as ImageNet format.
dataset_type = 'ImageNet'
img_norm_cfg = dict(
     mean=[124.508, 116.050, 106.438],
     std=[58.577, 57.310, 57.437],
     to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, -1), backend='pillow'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]
data = dict(
    # Specify the batch size and number of workers in each GPU.
    # Please configure it according to your hardware.
    samples_per_gpu=32,
    workers_per_gpu=2,
    # Specify the training dataset type and path
    train=dict(
        type=dataset_type,
        data_prefix='/content/drive/MyDrive/export/images/',
        classes='/content/drive/MyDrive/export/classes.txt',
        pipeline=train_pipeline),
    # Specify the validation dataset type and path
    val=dict(
        type=dataset_type,
        data_prefix='/content/drive/MyDrive/export/images/',
        ann_file='/content/drive/MyDrive/export/val_0829.txt',
        classes='/content/drive/MyDrive/export/classes.txt',
        pipeline=test_pipeline),
    # Specify the test dataset type and path
    test=dict(
        type=dataset_type,
        data_prefix='/content/drive/MyDrive/export/images/',
        ann_file='/content/drive/MyDrive/export/test_0829.txt',
        classes='/content/drive/MyDrive/export/classes.txt',
        pipeline=test_pipeline))

# Specify evaluation metric
evaluation = dict(metric='accuracy', metric_options={'topk': (1, )})

# ---- Schedule configs ----
# Usually in fine-tuning, we need a smaller learning rate and less training epochs.
# Specify the learning rate
optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=None)
# Set the learning rate scheduler
lr_config = dict(policy='step', step=1, gamma=0.1)
runner = dict(type='EpochBasedRunner', max_epochs=50)

# ---- Runtime configs ----
# Output training log every 10 iterations.
log_config = dict(interval=10)

Overwriting configs/mobilenet_v2/mobilenet_v2_1x_custom.py


In [ ]:
# Check MMClassification installation
import mmcls
print(mmcls.__version__)

In [ ]:
!python tools/train.py \
  configs/mobilenet_v2/mobilenet_v2_1x_custom.py \
  --work-dir work_dirs/mobilenet_v2_1x_custom \
  --seed 0 \
  --deterministic

/usr/local/lib/python3.7/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
/content/drive/MyDrive/mmclassification/mmcls/utils/setup_env.py:33: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/content/drive/MyDrive/mmclassification/mmcls/utils/setup_env.py:43: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment

KeyboardInterrupt: ignored

In [ ]:
!python tools/test.py configs/mobilenet_v2/mobilenet_v2_1x_custom.py /content/drive/MyDrive/mmclassification/work_dirs/cats_dogs_dataset/latest.pth --metrics accuracy --metric-options topk=1

/usr/local/lib/python3.7/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '
/content/drive/MyDrive/mmclassification/mmcls/utils/setup_env.py:33: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting OMP_NUM_THREADS environment variable for each process '
/content/drive/MyDrive/mmclassification/mmcls/utils/setup_env.py:43: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  f'Setting MKL_NUM_THREADS environment

AttributeError: ignored

AttributeError: ignored